# Librerias y carga de datos

In [ ]:
import numpy as np
import pandas as pd
from getpass import getpass
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm



df = pd.read_csv("/content/df.csv")

# Limpieza simple de los datos

In [ ]:
df['is_song'] = df['is_song'].replace({'No':'no'})
df['is_song'] = df['is_song'].replace({'Yes':'yes'})

In [ ]:
df['is_song'] = df['is_song'].replace({'yes': 1, 'no': 0})

print(df['is_song'].value_counts())

is_song
1    614
0    386
Name: count, dtype: int64


/tmp/ipython-input-2258975603.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['is_song'] = df['is_song'].replace({'yes': 1, 'no': 0})


In [ ]:

df["hashtags_used"] = (
    df["hashtags_used"]
    .fillna("")
    .str.replace(r"\s+", "", regex=True)
    .apply(lambda x: " ".join([f"#{tag}" for tag in x.split(",") if tag != ""]))
)


# Modelo facebook/nllb-200-distilled-1.3B

In [ ]:

model_id = "facebook/nllb-200-distilled-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cuda")


src_lang = "spa_Latn"
tgt_lang = "eng_Latn"


translated = []
for text in tqdm(df["descripcion"].fillna("").tolist()):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to("cuda")
    translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang))
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    translated.append(translated_text)

df["descripcion_en"] = translated


# Descargar archivo CSV

In [ ]:
df.to_csv("/content/df_traducido.csv", index=False)
from google.colab import files
files.download("/content/df_traducido.csv")

